<a href="https://colab.research.google.com/github/borja-izquierdo/portfolio/blob/main/Prophet_PYPL_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install Prophet and Yahoo Finance

In [15]:
# Instalamos el modelo predictivo Prophet
!python -m pip install prophet --quiet
# De aquí vamos a descargar los datos
!pip install yfinance --quiet


## Import necessary libraries

In [16]:
import pandas as pd
import yfinance as yf
import datetime as dt
import matplotlib.pyplot as plt
from datetime import timedelta
import plotly.graph_objects as go
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

## Set time from to a certain numbers of years

In [17]:
years = 10

endDate = dt.datetime.now()
startDate = endDate - dt.timedelta(days = 365*years)

## Create a ticker variable

In [32]:
ticker = 'PYPL'

## Download the daily data of the ticker

In [47]:
adj_close_df = pd.DataFrame()

adj_close_df =  yf.download(ticker, start = startDate, end = endDate).reset_index()

adj_close_df.tail()

[*********************100%%**********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
2100,2023-11-06,$56.65,$56.72,$54.19,$54.62,$54.62,17246800
2101,2023-11-07,$54.44,$55.04,$54.05,$54.63,$54.63,13038500
2102,2023-11-08,$54.33,$55.83,$53.55,$55.08,$55.08,20518500
2103,2023-11-09,$55.80,$56.03,$54.15,$54.28,$54.28,12568700
2104,2023-11-10,$54.34,$54.83,$53.87,$54.77,$54.77,9682100


## NAs validation

In [49]:
adj_close_df.isnull().sum()

ds        0
Open      0
High      0
Low       0
Close     0
y         0
Volume    0
dtype: int64

## Filter by columns 'Date' & 'Adj Close'

In [50]:
columns =['Date','Adj Close']

new_names = {
    'Date': 'ds',
    'Adj Close':'y',
}
adj_close_df.rename(columns = new_names, inplace=True)
adj_close_df['ds'] = adj_close_df['ds'].dt.tz_localize(None)

## Plot the Historical Adjusted Close Price

In [56]:
x = adj_close_df['ds']
y = adj_close_df['y']

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Le ponemos el título
fig.update_layout(
    title_text= f'{ticker} Time Series Adjusted Close Price',
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1a", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

# Training the model

## Initialize and training with the DataFrame


In [57]:
m= Prophet()
m= Prophet(seasonality_mode='multiplicative')

In [59]:
m.fit(adj_close_df)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmphqdpf2j9/jgjx2lbe.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmphqdpf2j9/u2ump0e1.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=99887', 'data', 'file=/tmp/tmphqdpf2j9/jgjx2lbe.json', 'init=/tmp/tmphqdpf2j9/u2ump0e1.json', 'output', 'file=/tmp/tmphqdpf2j9/prophet_modelijzl372m/prophet_model-20231113143732.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
14:37:32 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
14:37:36 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


### How many days we want to forecast

In [60]:
future = m.make_future_dataframe(60)
future.tail()

,ds
2160,2024-01-05
2161,2024-01-06
2162,2024-01-07
2163,2024-01-08
2164,2024-01-09


### Delete weekends for stocks, ETFs, Forex, Bonds and similars

In [61]:
future = future[ future['ds'].dt.dayofweek < 5 ]

### DataFrame to forecast

In [63]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
2142,2024-01-03,$56.43,$46.45,$67.30
2143,2024-01-04,$56.44,$46.39,$66.96
2144,2024-01-05,$56.38,$45.57,$67.28
2145,2024-01-08,$56.08,$45.16,$67.66
2146,2024-01-09,$55.99,$44.63,$66.62


In [70]:
next_day = (dt.date.today() + timedelta(days=1)).strftime('%Y-%m-%d')

forecast[forecast['ds'] == next_day]['yhat'].item()

54.40763205906773

## Forecast Plot

In [72]:
plot_plotly(m, forecast)

In [73]:
plot_components_plotly(m, forecast)